In [1]:
import mlrun

In [2]:
project = mlrun.get_or_create_project("compare-models", context="./")

> 2022-05-02 22:52:38,429 [info] created and saved project compare-models


### Log Dataset and Models

In [3]:
train_fn = mlrun.code_to_function(name="train", filename="train.py", kind="job", image="mlrun/ml-models:0.10.0")

In [4]:
log_test_set = train_fn.run(handler="log_test_set")

> 2022-05-02 22:52:53,828 [info] starting run train-log_test_set uid=f1d7d4fa352b49138e352f56113e9dbb DB=http://mlrun-api:8080
> 2022-05-02 22:52:53,997 [info] Job is running in the background, pod: train-log-test-set-4v6vq
> 2022-05-02 22:52:59,832 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
compare-models,...113e9dbb,0,May 02 22:52:57,completed,train-log_test_set,v3io_user=nickkind=jobowner=nickmlrun/client_version=0.10.0host=train-log-test-set-4v6vq,,,,X_testy_test


> 2022-05-02 22:53:00,856 [info] run executed, status=completed


In [5]:
train_reference = train_fn.run(handler="train_reference")

> 2022-05-02 22:53:00,860 [info] starting run train-train_reference uid=35bcaebec3674ed9acd1e859732bddc4 DB=http://mlrun-api:8080
> 2022-05-02 22:53:01,000 [info] Job is running in the background, pod: train-train-reference-hb4k6
2022-05-02 22:53:05.898863: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 22:53:05.900827: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-02 22:53:05.912593: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Epoch 1/5
1/1 [=======================

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
compare-models,...732bddc4,0,May 02 22:53:04,completed,train-train_reference,v3io_user=nickkind=jobowner=nickmlrun/client_version=0.10.0host=train-train-reference-hb4k6,,,lr=0.0010000000474974513training_loss=1.9061732292175293training_accuracy=0.33000001311302185validation_loss=1.8284940719604492validation_accuracy=0.3400000035762787,training_loss.htmltraining_accuracy.htmlvalidation_loss.htmlvalidation_accuracy.htmlloss_summary.htmlaccuracy_summary.htmllr_values.htmlmodel


> 2022-05-02 22:53:13,410 [info] run executed, status=completed


In [6]:
train_current = train_fn.run(handler="train_current")

> 2022-05-02 22:53:13,415 [info] starting run train-train_current uid=5afeab09a122427cb072b992f0f8694d DB=http://mlrun-api:8080
> 2022-05-02 22:53:13,592 [info] Job is running in the background, pod: train-train-current-7q652
2022-05-02 22:53:18.625624: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 22:53:18.627566: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-02 22:53:18.639458: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Epoch 1/5
1/1 [===========================

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
compare-models,...f0f8694d,0,May 02 22:53:16,completed,train-train_current,v3io_user=nickkind=jobowner=nickmlrun/client_version=0.10.0host=train-train-current-7q652,,,lr=0.0010000000474974513training_loss=1.0946334600448608training_accuracy=0.3199999928474426validation_loss=1.0913176536560059validation_accuracy=0.3400000035762787,training_loss.htmltraining_accuracy.htmlvalidation_loss.htmlvalidation_accuracy.htmlloss_summary.htmlaccuracy_summary.htmllr_values.htmlmodel


> 2022-05-02 22:53:26,054 [info] run executed, status=completed


### Build Comparison Function

In [7]:
compare_models_fn = mlrun.code_to_function(
    name="compare-automlrun-models",
    filename="compare_automlrun_models.py",
    kind="job",
    handler="compare_models",
    with_doc=False,
    image="mlrun/ml-models:0.10.0",
    requirements=["evidently==0.1.49.dev0"]
).apply(mlrun.auto_mount())

In [8]:
compare_models_fn.export()

> 2022-05-02 22:53:26,178 [info] function spec saved to path: function.yaml


In [9]:
compare_models_fn.deploy()

> 2022-05-02 22:53:26,285 [info] Started building image: .mlrun/func-compare-models-compare-automlrun-models:latest
INFO[0000] Retrieving image manifest mlrun/ml-models:0.10.0 
INFO[0000] Retrieving image mlrun/ml-models:0.10.0 from registry index.docker.io 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest mlrun/ml-models:0.10.0 
INFO[0000] Returning cached image manifest              
INFO[0000] Executing 0 build triggers                   
INFO[0000] Unpacking rootfs as cmd RUN python -m pip install evidently==0.1.49.dev0 requires it. 
INFO[0063] RUN python -m pip install evidently==0.1.49.dev0 
INFO[0063] Taking snapshot of full filesystem...        
INFO[0097] cmd: /bin/sh                                 
INFO[0097] args: [-c python -m pip install evidently==0.1.49.dev0] 
INFO[0097] Running: [/bin/sh -c python -m pip install evidently==0.1.49.dev0] 
INFO[0100] Taking snapshot of full filesystem...        
INFO[0105] Pushing image to dock

True

### Compare Models

In [10]:
run = compare_models_fn.run(
    inputs={
        "reference_model_path" : train_reference.outputs["model"],
        "current_model_path" : train_current.outputs["model"],
        "X_test_path" : log_test_set.outputs["X_test"],
        "y_test_path" : log_test_set.outputs["y_test"]
    },
    params={
        "class_mappings": {0 : 'setosa', 1 : 'versicolor', 2 : 'virginica'},
        "target": "target",
        "comparison_metric": "accuracy"
    }
)

> 2022-05-02 22:55:15,802 [info] starting run compare-automlrun-models-compare_models uid=c00c8cbb04194c148c284905fbdc09c0 DB=http://mlrun-api:8080
> 2022-05-02 22:55:15,966 [info] Job is running in the background, pod: compare-automlrun-models-compare-models-4frr9
2022-05-02 22:55:26.277901: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 22:55:26.279977: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
> 2022-05-02 22:55:27,474 [info] run executed, status=completed
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
compare-models,...fbdc09c0,0,May 02 22:55:23,completed,compare-automlrun-models-compare_models,v3io_user=nickkind=jobowner=nickmlrun/client_version=0.10.0host=compare-automlrun-models-compare-models-4frr9,reference_model_pathcurrent_model_pathX_test_pathy_test_path,"class_mappings={'0': 'setosa', '1': 'versicolor', '2': 'virginica'}target=targetcomparison_metric=accuracy",reference_accuracy=0.34reference_precision=0.11333333333333334reference_recall=0.3333333333333333reference_f1=0.1691542288557214reference_roc_auc=0.5406454248366013reference_log_loss=1.8284942139685154current_accuracy=0.34current_precision=0.12056737588652483current_recall=0.3333333333333333current_f1=0.17708333333333334current_roc_auc=0.8223410576351752current_log_loss=1.0913175964355468best_model=reference,prob_classification_dashboard


> 2022-05-02 22:55:35,331 [info] run executed, status=completed
